# MTA Analysis: Data Acquisition & Cleaning
---

### Setup
#### Import required libraries.

In [1]:
import pandas as pd
import numpy as np
import os

#### Define useful functions (maybe document and put this in .py file later?)

In [29]:
def convert_timestamp_to_mta_format(timestamp):
    """
    """
    year = str(timestamp.year)[2:4]
    month = str(timestamp.month).zfill(2)
    day = str(timestamp.day).zfill(2)
    
    date_mta_format = year + month + day
    
    return date_mta_format

def calculate_hourly_turnstile_traffic(turnstile_data_row, reset_limit, entries=True):
    """
    Calculates hourly turnstile traffic (entries or exits) using current day and previous day readings.
    Adjusts for erroneous reverse counting and turnstile count resets.
    
    :param pandas.Series turnstile_data_row: Row of data corresponding to a single 4-hour span of 
                                             data for a single turnstile.
    :param int reset_limit: Used as an upper limit to determine if turnstile reset.
    :return: Total hourly traffic (entries or exits) for a single turnstile.
    :rtype: int
    """
    traffic_type = 'ENTRIES' if entries else 'EXITS'
    
    # calculate initial hourly traffic
    hourly_traffic = turnstile_data_row[traffic_type] - turnstile_data_row["PREV_" + traffic_type]
    
    # reverse sign if turnstile was counting backwards erroneously
    hourly_traffic = -hourly_traffic if hourly_traffic < 0 else hourly_traffic

    # if counter was possibly reset to 0
    if hourly_traffic > reset_limit:
        hourly_traffic = min(turnstile_data_row[traffic_type], turnstile_data_row["PREV_" + traffic_type])

    # if the reset limit is reached, reset entries to 0     
    if hourly_traffic > reset_limit:
        hourly_traffic = 0

    return hourly_traffic

### Data Acquisition
Load [MTA Turnstile Data](http://web.mta.info/developers/turnstile.html) and combine into a single dataframe.
_Note: The files already exist in the repository. This code block can be skipped over._

In [ ]:
start_date = '2015-05-02'
months_of_interest = [5, 6]
start_ts = pd.Timestamp(start_date)

In [ ]:
current_timestamp = start_ts
is_date_valid = True
raw_master_df = pd.DataFrame()
while is_date_valid:
    if current_timestamp.month not in months_of_interest:
        current_timestamp += pd.DateOffset(days=7)
        continue
    
    print(f'Downloading data for {current_timestamp}...')
    date_formatted = convert_timestamp_to_mta_format(current_timestamp)
    
    # load data and write to csv
    url = f'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{date_formatted}.txt'
    df_turnstile_data = pd.read_csv(url)
    raw_master_df = pd.concat([raw_master_df, df_turnstile_data])
    
    # add 7 days to get next file. if resulting date is later than today, then stop loop
    current_timestamp += pd.DateOffset(days=7)
    if current_timestamp > pd.to_datetime("now"):
        is_date_valid = False

Export combined dataframe to pickle.

In [ ]:
raw_master_df.to_pickle('raw_mta_turnstile_data_mayjune_20152020.pkl')

### Data Cleaning
Load pickled data.

In [3]:
df_mta_raw = pd.read_pickle('raw_mta_turnstile_data_mayjune_20152020.pkl')
df_mta_raw.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858


Remove any whitespace from column names and initialize a new dataframe to store cleaned data.

In [4]:
df_mta_clean = df_mta_raw
df_mta_clean.columns = df_mta_clean.columns.str.replace(' ', '')

Determine if there are any NaN values.

In [5]:
df_mta_raw[df_mta_clean.isna()].count()

C/A         0
UNIT        0
SCP         0
STATION     0
LINENAME    0
DIVISION    0
DATE        0
TIME        0
DESC        0
ENTRIES     0
EXITS       0
dtype: int64

No NaN values to filter.

Add `DATETIME` column to dataset.

In [6]:
df_mta_clean['DATETIME'] = pd.to_datetime(df_mta_clean['DATE'] + ' ' + df_mta_clean['TIME'])

Check if there are duplicates by determining if there are multiple readings for a given turnstile at a given time.

In [7]:
df_checking_duplicates = (df_mta_clean.groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"]).ENTRIES.count().reset_index()).sort_values('ENTRIES')
df_checking_duplicates

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2016-04-30 00:00:00,1
6896126,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 04:00:00,1
6896127,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 08:00:00,1
6896128,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 12:00:00,1
6896129,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 16:00:00,1
...,...,...,...,...,...,...
6410336,R101,R001,02-00-00,SOUTH FERRY,2018-05-23 09:00:00,2
2073816,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-14 05:00:00,2
4774408,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-16 16:00:00,2
4774415,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-17 08:00:00,2


In [8]:
df_checking_duplicates = df_checking_duplicates[df_checking_duplicates['ENTRIES'] == 2]
df_checking_duplicates

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
9359628,R518,R261,00-03-02,40 ST LOWERY ST,2017-06-05 08:00:00,2
5189785,N525,R142,01-00-03,DELANCEY/ESSEX,2019-05-11 05:00:00,2
811622,B020,R263,00-06-01,AVENUE H,2016-05-30 12:00:00,2
4290068,N329,R201,00-03-04,WOODHAVEN BLVD,2015-05-20 05:00:00,2
8513239,R290,R161,00-00-00,KINGSBRIDGE RD,2019-06-07 05:00:00,2
...,...,...,...,...,...,...
6410336,R101,R001,02-00-00,SOUTH FERRY,2018-05-23 09:00:00,2
2073816,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-14 05:00:00,2
4774408,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-16 16:00:00,2
4774415,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-17 08:00:00,2


In [9]:
# TODO use a merge to get relevant rows from original df

Check to see what is causing the duplicate.

In [10]:
mask = (
    (df_mta_clean['C/A'] == 'R518') &
    (df_mta_clean['UNIT'] == 'R261') &
    (df_mta_clean['SCP'] == '00-03-02') &
    (df_mta_clean['STATION'] == '40 ST LOWERY ST') &
    (df_mta_clean['DATETIME'] == pd.to_datetime('2017-06-05 08:00:00'))
)
df_mta_clean[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
178845,R518,R261,00-03-02,40 ST LOWERY ST,7,IRT,06/05/2017,08:00:00,REGULAR,18778966,7453831,2017-06-05 08:00:00
178846,R518,R261,00-03-02,40 ST LOWERY ST,7,IRT,06/05/2017,08:00:00,RECOVR AUD,18778965,7453831,2017-06-05 08:00:00


Spot check randomly to see if REGULAR and RECOVR AUD readings are different.

In [11]:
mask = (
    (df_mta_clean['C/A'] == 'R290') &
    (df_mta_clean['UNIT'] == 'R161') &
    (df_mta_clean['SCP'] == '00-00-00') &
    (df_mta_clean['STATION'] == 'KINGSBRIDGE RD') &
    (df_mta_clean['DATETIME'] == pd.to_datetime('2019-06-07 05:00:00'))
)
df_mta_clean[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
169384,R290,R161,00-00-00,KINGSBRIDGE RD,4,IRT,06/07/2019,05:00:00,REGULAR,183445,164296,2019-06-07 05:00:00
169385,R290,R161,00-00-00,KINGSBRIDGE RD,4,IRT,06/07/2019,05:00:00,RECOVR AUD,183444,164296,2019-06-07 05:00:00


In [12]:
mask = (
    (df_mta_clean['C/A'] == 'N045') &
    (df_mta_clean['UNIT'] == 'R187') &
    (df_mta_clean['SCP'] == '01-00-02') &
    (df_mta_clean['STATION'] == '81 ST-MUSEUM') &
    (df_mta_clean['DATETIME'] == pd.to_datetime('2019-06-17 09:00:00'))
)
df_mta_clean[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
49121,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/17/2019,09:00:00,REGULAR,4343886,994027,2019-06-17 09:00:00
49122,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/17/2019,09:00:00,RECOVR AUD,4343885,994027,2019-06-17 09:00:00


In [13]:
df_mta_clean.DESC.value_counts()

REGULAR       10303603
RECOVR AUD       40700
Name: DESC, dtype: int64

Based on a spot check, it appears that where there are duplicates, the REGULAR and RECOVR AUD readings are identical.
We will assume that we can drop any duplicate entries (regardless of the type).

In [14]:
df_mta_clean.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)

Confirm no duplicates.

In [15]:
(df_mta_clean.groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"]).ENTRIES.count().reset_index()).sort_values('ENTRIES')

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2016-04-30 00:00:00,1
6896138,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 04:00:00,1
6896139,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 08:00:00,1
6896140,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 12:00:00,1
6896141,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 16:00:00,1
...,...,...,...,...,...,...
3448070,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-25 17:00:00,1
3448071,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-25 21:00:00,1
3448072,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-26 01:00:00,1
3448065,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-24 21:00:00,1


Drop columns `DESC`. Supress errors.

In [16]:
df_mta_clean = df_mta_clean.drop("DESC", axis=1, errors="ignore")
df_mta_clean.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,5106770,1729635,2015-04-25 00:00:00
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,5106810,1729649,2015-04-25 04:00:00
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,5106835,1729680,2015-04-25 08:00:00
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,5106961,1729784,2015-04-25 12:00:00
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,5107250,1729858,2015-04-25 16:00:00


Turnstile counts (entries and exits) are cumulative. In order to actually calculate the number of entries/exits per timestamp, some calculations need to be performed.

In [17]:
df_mta_clean.reset_index(inplace=True)
df_mta_clean[["PREV_DATETIME", "PREV_ENTRIES", "PREV_EXITS"]] = (
    df_mta_clean
    .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATETIME", "ENTRIES", "EXITS"]
    .apply(lambda grp: grp.shift(1)))

df_mta_clean.head()

<ipython-input-17-e48f0495b2aa>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mta_clean


,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS
0,0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,5106770,1729635,2015-04-25 00:00:00,NaT,NaN,NaN
1,1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,5106810,1729649,2015-04-25 04:00:00,2015-04-25 00:00:00,5106770.0,1729635.0
2,2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,5106835,1729680,2015-04-25 08:00:00,2015-04-25 04:00:00,5106810.0,1729649.0
3,3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,5106961,1729784,2015-04-25 12:00:00,2015-04-25 08:00:00,5106835.0,1729680.0
4,4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,5107250,1729858,2015-04-25 16:00:00,2015-04-25 12:00:00,5106961.0,1729784.0


In [18]:
df_mta_clean.drop('index', axis=1, inplace=True)

Since there is no hour previous to the first hour, this resulted in `NaN` for the first hour of data for each turnstile. Those rows should be dropped. This will also take care of any other `NaN` data.

In [19]:
df_mta_clean.dropna(axis=0, inplace=True)
df_mta_clean.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,5106810,1729649,2015-04-25 04:00:00,2015-04-25 00:00:00,5106770.0,1729635.0
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,5106835,1729680,2015-04-25 08:00:00,2015-04-25 04:00:00,5106810.0,1729649.0
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,5106961,1729784,2015-04-25 12:00:00,2015-04-25 08:00:00,5106835.0,1729680.0
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,5107250,1729858,2015-04-25 16:00:00,2015-04-25 12:00:00,5106961.0,1729784.0
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,20:00:00,5107620,1729914,2015-04-25 20:00:00,2015-04-25 16:00:00,5107250.0,1729858.0


Calculates hourly entries and exits for each 4-hour timestamp.

In [30]:
df_mta_clean['HOURLY_ENTRIES'] = df_mta_clean.apply(calculate_hourly_turnstile_traffic, axis=1, reset_limit=4000)
df_mta_clean.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,HOURLY_ENTRIES
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,5106810,1729649,2015-04-25 04:00:00,2015-04-25 00:00:00,5106770.0,1729635.0,40.0
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,5106835,1729680,2015-04-25 08:00:00,2015-04-25 04:00:00,5106810.0,1729649.0,25.0
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,5106961,1729784,2015-04-25 12:00:00,2015-04-25 08:00:00,5106835.0,1729680.0,126.0
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,5107250,1729858,2015-04-25 16:00:00,2015-04-25 12:00:00,5106961.0,1729784.0,289.0
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,20:00:00,5107620,1729914,2015-04-25 20:00:00,2015-04-25 16:00:00,5107250.0,1729858.0,370.0


In [31]:
df_mta_clean['HOURLY_EXITS'] = df_mta_clean.apply(calculate_hourly_turnstile_traffic, axis=1, reset_limit=4000, 
                                                  entries=False)
df_mta_clean.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,HOURLY_ENTRIES,HOURLY_EXITS
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,5106810,1729649,2015-04-25 04:00:00,2015-04-25 00:00:00,5106770.0,1729635.0,40.0,14.0
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,5106835,1729680,2015-04-25 08:00:00,2015-04-25 04:00:00,5106810.0,1729649.0,25.0,31.0
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,5106961,1729784,2015-04-25 12:00:00,2015-04-25 08:00:00,5106835.0,1729680.0,126.0,104.0
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,5107250,1729858,2015-04-25 16:00:00,2015-04-25 12:00:00,5106961.0,1729784.0,289.0,74.0
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,20:00:00,5107620,1729914,2015-04-25 20:00:00,2015-04-25 16:00:00,5107250.0,1729858.0,370.0,56.0


In [32]:
df_mta_clean.describe(include='all')

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,HOURLY_ENTRIES,HOURLY_EXITS
count,10337013,10337013,10337013,10337013,10337013,10337013,10337013,10337013,1.033701e+07,1.033701e+07,10337013,10337013,1.033701e+07,1.033701e+07,1.033701e+07,1.033701e+07
unique,752,471,242,572,128,6,364,85794,NaN,NaN,658092,658091,NaN,NaN,NaN,NaN
top,PTH22,R549,00-00-00,34 ST-PENN STA,1,IRT,04/29/2019,04:00:00,NaN,NaN,2020-06-03 04:00:00,2020-06-03 04:00:00,NaN,NaN,NaN,NaN
freq,62204,140593,1003801,219226,1245519,3763399,30972,885494,NaN,NaN,2538,2538,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-04-25 04:00:00,2015-04-25 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-26 23:58:45,2020-06-26 19:46:45,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.825125e+07,3.110195e+07,NaN,NaN,3.824471e+07,3.109653e+07,1.547153e+02,1.201396e+02
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.042166e+08,1.855490e+08,NaN,NaN,2.041979e+08,1.855317e+08,2.469555e+02,2.194078e+02
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.461090e+05,2.068710e+05,NaN,NaN,4.461050e+05,2.068830e+05,5.000000e+00,5.000000e+00


In [33]:
df_mta_clean.to_pickle('cleaned_mta_turnstile_data_mayjune_20152020.pkl')